# SDXL Inpainting - Modern High-Quality Solution

Better than LaMa with built-in refinement. Works with current Colab environment.

In [ ]:
# Install dependencies (works with latest Colab)
!pip install diffusers transformers accelerate safetensors

In [ ]:
from diffusers import AutoPipelineForInpainting
import torch
from PIL import Image
import numpy as np

# Load SDXL inpainting pipeline
pipe = AutoPipelineForInpainting.from_pretrained(
    "diffusers/stable-diffusion-xl-1.0-inpainting-0.1",
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

# Enable memory efficient attention
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
# Upload your files
from google.colab import files
import zipfile

uploaded = files.upload()
zip_name = list(uploaded.keys())[0]

with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall('.')

# Load images
image = Image.open("image.png").convert("RGB")
mask = Image.open("mask.png").convert("L")

In [ ]:
# High-quality inpainting with automatic refinement
result = pipe(
    prompt="high quality interior, professional photography",
    negative_prompt="blurry, artifacts, distorted",
    image=image,
    mask_image=mask,
    num_inference_steps=50,  # More steps = better quality
    strength=0.99,
    guidance_scale=8.0,
    height=image.height,
    width=image.width
).images[0]

# Display results
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(image)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(mask, cmap='gray')
axes[1].set_title('Mask')
axes[1].axis('off')

axes[2].imshow(result)
axes[2].set_title('SDXL Inpainted')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# Save and download
result.save("result_sdxl.png")
files.download("result_sdxl.png")